In [1]:
import requests
from bs4 import BeautifulSoup
import ccxt
from tqdm.notebook import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time

In [2]:
url = 'https://coinmarketcap.com/exchanges/binance/'
response = requests.get(url)
content = response.text

In [3]:
soup = BeautifulSoup(content, 'html.parser')

In [4]:
market_data_class = 'sc-7b3ac367-3 NFswn cmc-table'
table = soup.find(class_=market_data_class)

In [5]:
column_index = 2

symbols = []

for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > column_index:
        symbols.append(cells[column_index].text)

In [6]:
binance = ccxt.binance()

In [9]:
def fetch_ohlcv(symbol, interval, start_timestamp):
    from_ts = binance.parse8601(start_timestamp)
    ohlcv = binance.fetch_ohlcv(symbol, interval, since=from_ts, limit=1000)
    while True:
        from_ts = ohlcv[-1][0] + 1
        new_ohlcv = binance.fetch_ohlcv(symbol, interval, since=from_ts, limit=1000)
        ohlcv.extend(new_ohlcv)
        if len(new_ohlcv) != 1000:
            break
    data_dict = {symbol: ohlcv}
    return data_dict

In [10]:
start_timestamp = '2024-06-21 19:15:00'
interval = '15m'
full_ohlcv = {}

for symbol in tqdm(symbols, desc='Fetching OHLCV Data', unit='trading pair'):
    ohlcv = fetch_ohlcv(symbol, interval, start_timestamp)
    full_ohlcv.update(ohlcv)

Fetching OHLCV Data:   0%|          | 0/50 [00:00<?, ?trading pair/s]

In [20]:
data_for_df = []

for symbol, ohlcv in full_ohlcv.items():
    for entry in ohlcv:
        data_for_df.append([entry[0], symbol] + entry[1:])

In [26]:
df = pd.DataFrame(data_for_df, columns=['Timestamp', 'Pair', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
df.head()

,Timestamp,Pair,Open,High,Low,Close,Volume
0,2024-06-21 19:15:00,BTC/FDUSD,63942.02,64072.13,63933.64,64004.01,319.60860
1,2024-06-21 19:30:00,BTC/FDUSD,64002.94,64133.57,63998.18,64115.95,318.18427
2,2024-06-21 19:45:00,BTC/FDUSD,64114.12,64240.00,64052.89,64197.99,371.73904
3,2024-06-21 20:00:00,BTC/FDUSD,64196.03,64374.00,64169.85,64249.50,409.04486
4,2024-06-21 20:15:00,BTC/FDUSD,64251.91,64435.48,64232.43,64235.30,271.01192
